In [ ]:
!pip install -r requirements.txt

: 

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
import warnings
import os
from datetime import datetime
from dotenv import load_dotenv
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from langchain_community.llms import VLLMOpenAI

# Ignore SyntaxWarning from pysbd
warnings.filterwarnings("ignore", category=SyntaxWarning, module="pysbd")

In [ ]:
load_dotenv()

# Language model configuration
llm = VLLMOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    model="openai/hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
    temperature=0,
    max_tokens=400,
)

In [ ]:
from crewai import Agent

search_agent = Agent(
    role="{topic} Search Agent",
    goal="Search for the latest articles related to the given topic: {topic}.",
    backstory=(
        "As a Search Agent, you're an expert at gathering relevant information "
        "from the web on a specific topic. Your mission is to retrieve recent, "
        "popular articles that are related to the subject of interest."
    ),
    llm=llm,
    verbose=True,
)

analysis_agent = Agent(
    role="{topic} Analysis Agent",
    goal="Analyze and rank the relevance and popularity of the found articles.",
    backstory=(
        "As an Analysis Agent, you're skilled in identifying important trends "
        "and analyzing articles based on their content and popularity."
    ),
    llm=llm,
    verbose=True,
)

report_agent = Agent(
    role="{topic} Report Agent",
    goal="Summarize the findings into a final user-facing report.",
    backstory=(
        "As a Report Agent, you're a professional summarizer. Your task is to "
        "craft a concise, informative report based on the findings of the other agents."
    ),
    llm=llm,
    verbose=True,
)


In [ ]:
def create_tasks():
    # Create tasks directly with their respective agents
    search_task = Task(
        description=(
            "Search for the latest and most relevant articles about the topic: {topic}. "
            "Use reputable sources such as news outlets, industry blogs, and trend analysis platforms. "
            "Extract meaningful content — not just links — including key quotes or summaries."
        ),
        expected_output=(
            "A curated list of the top 2 specific and up-to-date articles related to {topic}, each including:\n"
            "- Article title\n"
            "- Source name\n"
            "- Publication date\n"
            "- Direct link to the article page (not homepage)\n"
            "- 3 sentence summary of the article content"
        ),
        agent=search_agent
    )
    
    analysis_task = Task(
        description=(
            "Given a list of articles on a topic, the agent must analyze each article and rank them in order of relevance to the original search topic.\n"
            "For each article, the agent should provide:\n"
            "- Title\n"
            "- A short summary explaining why the article is relevant to the search topic\n\n"
            "The output should be a ranked list (starting from most to least relevant), with both articles analyzed thoroughly.\n"
            "Do not skip any article. Assume the list always includes exactly 2 articles.\n"
            "The analysis should be objective and based on content relevance."
        ),
        expected_output=(
            "A ranked list of the top 2 articles, each with a short summary"
        ),
        agent=analysis_agent,
        dependencies=[search_task]
    )
    
    report_task = Task(
        description=(
            "Using the results from the analysis task, write a structured and concise report summarizing the top trends related to {topic}."
        ),
        expected_output=(
            "A structured trend report highlighting the key findings, written in markdown format."
        ),
        agent=report_agent,
        dependencies=[search_task, analysis_task],
        output_file="trend_report.md"
    )
    
    return [search_task, analysis_task, report_task]


In [ ]:
def run_trend_hunter():
    """Run the Trend Hunter crew."""
    print("Starting Trend Hunter App with OpenShift vLLM backend...\n")
    print("## Welcome to the Trend Hunter!")
    print("--------------------------------------")

    topic = input("Enter the trend you want to track:\n")
    inputs = {
        "topic": topic,
        "current_year": str(datetime.now().year),
    }

    try:
        # Create tasks
        tasks = create_tasks()
        
        # Create crew with tasks
        crew = Crew(
            tasks=tasks,
            process=Process.sequential,
            verbose=False,
        )
        
        # Run crew
        result = crew.kickoff(inputs=inputs)
        return result
    except Exception as e:
        print(f"Error details: {str(e)}")
        raise Exception(f"An error occurred while running the crew: {e}")

In [ ]:
# Run the application
try:
    result = run_trend_hunter()
    print(result)
except Exception as e:
    print(f"Failed to run: {e}")